In [ ]:
# Standard scientific Python imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Load the digits dataset
digits = datasets.load_digits()

In [ ]:
# Inspect dataset
SAMPLE = 0
(ROW, COL) = (5, 5)
(WIDTH, HEIGHT) = np.shape(digits.images[SAMPLE])

plt.figure(figsize=(5,5))
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:ROW*COL]):
    plt.subplot(ROW, COL, index + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
plt.show()

print(f'{digits.target[SAMPLE]} -> {WIDTH} x {HEIGHT}\n {digits.images[SAMPLE]}')

In [ ]:
# Convert categorical variable into dummy/indicator variables
labels = digits.target.reshape(len(digits.target),1) 
enc = OneHotEncoder()
enc.fit(labels)
onehotlabels = enc.transform(labels).toarray()
print(f'{digits.target[SAMPLE]} -> {onehotlabels[SAMPLE]}')

In [ ]:
wrapper_images = [np.reshape(i, (WIDTH*HEIGHT)) for i in digits.images]
wrapper_labels = onehotlabels

print(f'images -> {np.shape(wrapper_images)}, labels -> {np.shape(wrapper_labels)}')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(wrapper_images, wrapper_labels, test_size=0.33, random_state=42)
# convert matrix to (samples x features)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
class NeuralNetwork:
    """ Feedforward neural network """
    def __init__(self, neurons):
        self._layers = len(neurons)
        self._weights = [np.random.randn(nex, pre+1) for pre, nex in zip(neurons[:-1], neurons[1:])]
    
    def _init_params(self, x, y, iterations, learning_rate):
        """ Initilize parameters. """
        self._x = x.T
        self._y = y.T
        self._learning_rate = learning_rate
        self.costs_ = np.zeros(iterations)
        
    def _sigmoid(self, x):
        """ Computes sigmoid function. """
        return 1.0/(1 + np.exp(-x))    
    
    def _feedforward(self, a):  
        a_s = []
        for w in self._weights:
            a = np.vstack([np.ones((1, a.shape[1])), a])
            a_s.append(a)
            z = w.dot(a)
            a = self._sigmoid(z)
        a_s.append(a)
        return a_s
        
    def _backprop(self, a_s):
        """ Update weights. """
        delta_weights = [np.zeros(w.shape) for w in self._weights]
        # Update last layer delta
        delta = a_s[-1] - self._y
        delta_weights[-1] = delta.dot(a_s[-2].T)
        # Update all but the last layer delta
        for L in range(2, self._layers):
            # [1:] Ignore bias term
            delta = (self._weights[-L+1].T.dot(delta)*a_s[-L]*(1-a_s[-L]))[1:]
            delta_weights[-L] = delta.dot(a_s[-L-1].T) 
        # Update all weights
        self._weights = [w - self._learning_rate * dw for w, dw in zip(self._weights, delta_weights)]
    
    def _get_cost(self, a):
        """ Compute loss. """
        return 1/(2*self._y.shape[1]) * np.sum((a - self._y)**2)

    def fit(self, x, y, iterations=1000, learning_rate=0.5):
        self._init_params(x, y, iterations, learning_rate)

        for i in range(iterations):
            a_s = self._feedforward(self._x)
            self._backprop(a_s)
            self.costs_[i] = self._get_cost(a_s[-1])
        return self
    
    def predict(self, x):
        a_s = self._feedforward(x.T)
        return a_s[-1]
    
    def score(self, y_pred, y_test):
        return sum(int(np.argmax(x) == np.argmax(y)) for (x, y) in zip(y_pred.T, y_test.T)) / y_pred.shape[1]

In [ ]:
# Train model
neurons = [64, 64, 64, 10]
nn = NeuralNetwork(neurons)
model = nn.fit(X_train, y_train, iterations=5000, learning_rate=0.001)

In [ ]:
y_pred = nn.predict(X_test)
print(f'score: {nn.score(y_pred, y_test.T):.2f}')

In [ ]:
plt.figure(figsize=(10,5))
ax = plt.gca()
ax.grid(color='#b7b7b7', linestyle='-', linewidth=0.5, alpha=0.5)
plt.plot(nn.costs_, label='costs', color='#121212', linewidth=2, alpha=0.9)
plt.show()